# ODE Review

This tutorial demonstrates how to use `Ozone` to solve an ODE of the form
$$
\frac{\partial{y}}{\partial{t}} = f(y), 
$$
where $y$ is the state variable to integrate, with initial conditions $y_0$.

## Simple ODE
Let's solve the simple ODE:
$$ \frac{\partial{y}}{\partial{t}} = 0.5y-3, \qquad y_0 = 1$$

We'll solve this ODE from $t = 0$ to $t=1$ with a timestep size of $0.1$

To implement this in `ozone`, we formulate the ODE problem using `ozone`s `ODEProblem`. The `ODEProblem` object lets you specify:
- ODE states (just $y$ in this case)
- any ODE parameters (none in this example)
- numerical integration timespan (10 steps with size 0.1 for this example)
- numerical method
- solution approach

Start by importing the `ODEProblem` object and instantiate it by passing in the desired method and approach using `ozone`'s API. We'll use the impliit Gauss-Legendre 4 method and solve it using time-marching.

In [6]:
import numpy as np
import csdl_alpha as csdl

 # Start CSDL recorder
recorder = csdl.Recorder(inline = True)
recorder.start()

# Set a method and approach
import ozone
ode_problem = ozone.ODEProblem(
    ozone.methods.GaussLegendre4(),
    ozone.approaches.TimeMarching(),
)

In `ozone`, you can combine any method with any solution approach. Methods include the following explicit and implicit Runge-Kutta methods: 

- **Explicit**: ForwardEuler, ExplicitMidpoint, KuttaThirdOrder, RK4, RK6, RalstonsMethod, and HeunsMethod
- **Implicit**: BackwardEuler, ImplicitMidpoint, GaussLegendre2, GaussLegendre4, GaussLegendre6, Lobatto2, Lobatto4, RadauI3, RadauI5, RadauII3, RadauII5, Trapezoidal, AB1, AM1, and BDF1

These methods can be solved using one of four solution approaches:
- **TimeMarching**: Compute the state sequentially through timesteps.
- **PicardIteration**: Compute the state across timesteps in parallel
- **TimeMarchingUniform**: Same as time-marching but memory usage is reduced with the added cost of slower computation time
- **Collocation**: Solves for the state through an optimization problem

Let's continue by setting our ODE states. To define a state, provide a unique name and pass in a `CSDL` or NumPy variable for the initial condition. The state variable's shape will be inferred from the initial condition shape.

Here we set the initial condition $y_0 = 1.0$ and store the solution history as an output.

In [7]:
y_initial = csdl.Variable(value = 1.0) # Differentiable
ode_problem.add_state('y', y_initial, store_history=True)

Next, lets define the integration timespan. `Ozone` provides an API to help discretize the integration timespan for your problem using the `ozone.timespan` module. Lets define our timespan by giving a vector of timestep sizes. 

In this case, we define a uniform step size of 0.1 over 10 timesteps, integrating the system over the interval [0,1].

In [8]:
step_vector = csdl.Variable(value = np.full(10,0.1)) # Differentiable
ode_problem.set_timespan(
    ozone.timespans.StepVector(start=0.0, step_vector=step_vector)
)

Then, lets define the ODE function itself and add it to our `ODEProblem`. The ODE function must define state derivatives $\frac{\partial{y}}{\partial{t}}$ given $y$ using the unique name defined earlier.

For efficiency, the ODE function must be **vectorized**. This means it should handle the input $y$ and return the state derivatives in a vectorized form, which can significantly speed up the computation when working with large systems. `Ozone` will automatically set the shape of the state to be ($n$, (state shape)) and the user must return the derivatives with the same shape. For Gauss-Legendre 4 with time-marching, `ozone` automatically sets $n$ as 2.

For this problem, the ODE function looks like:

In [9]:
def ode_function(ozone_vars:ozone.ODEVars):
    y = ozone_vars.states['y'] # y shape is (2,1)
    print('y shape:', y.shape)
    ozone_vars.d_states['y'] = 0.5*y-3 # dy_dt = 0.5y-3
ode_problem.set_function(ode_function)

Finally, lets solve our ODE by calling the `solve` method.

In [10]:
ode_outputs = ode_problem.solve()
solved_y = ode_outputs.states['y']
print('numerically integrated y at t = 0, 0.1 , ... 1.0:')
print(solved_y.value)
print('analytical y at t = 1.0:')
print(6-5*np.exp(0.5))

y shape: (2, 1)
nonlinear solver: ozone_stage_solver converged in 1 iterations.
nonlinear solver: ozone_stage_solver converged in 1 iterations.
nonlinear solver: ozone_stage_solver converged in 1 iterations.
nonlinear solver: ozone_stage_solver converged in 1 iterations.
nonlinear solver: ozone_stage_solver converged in 1 iterations.
nonlinear solver: ozone_stage_solver converged in 1 iterations.
nonlinear solver: ozone_stage_solver converged in 1 iterations.
nonlinear solver: ozone_stage_solver converged in 1 iterations.
nonlinear solver: ozone_stage_solver converged in 1 iterations.
nonlinear solver: ozone_stage_solver converged in 1 iterations.
nonlinear solver: ozone_stage_solver converged in 1 iterations.
numerically integrated y at t = 0, 0.1 , ... 1.0:
[[ 1.        ]
 [ 0.74364452]
 [ 0.47414541]
 [ 0.19082879]
 [-0.10701378]
 [-0.42012707]
 [-0.74929402]
 [-1.09533772]
 [-1.45912346]
 [-1.8415609 ]
 [-2.24360632]]
analytical y at t = 1.0:
-2.243606353500642


This tutorial covered the basic workflow of defining and solving an ODE problem using `ozone`. To see a more advanced example with static parameters, dynamic parameters, and derivatives through the numerical integration process, refer to the Lotka-Volterra example in the same directory.